Windwerte und Niederschlag historisch und aktuell aus https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivstunden.html

* stundenwerte_FF_02014_akt.zip (Wind, Hannover, aktuell)
* stundenwerte_FF_02014_hist.zip (Wind, Hannover, historisch)
* stundenwerte_RR_02014_akt.zip (Niederschlag, Hannover, aktuell)
* stundenwerte_RR_02014_hist.zip (Niederschlag, Hannover, historisch)

In [4]:
import os
from dotenv import load_dotenv
from zipfile import ZipFile 
import pandas as pd

load_dotenv()
 
existing_files = os.listdir(os.environ['DWD_DATA'])
Hannover_code = '02014'
raw_dataframes = {}
for measure in ['FF','RR']:
    raw_dataframes[measure] = {}
    for section in ['akt','hist']:
        f = f'stundenwerte_{measure}_{Hannover_code}_{section}.zip'
        assert(f in existing_files)
        with ZipFile(f"{os.environ['DWD_DATA']}/{f}") as zip:
            prod_list = [df for df in zip.namelist() if df.startswith('produkt_')]
            assert(len(prod_list)==1)
            with zip.open(prod_list[0]) as product_file:
                df =  pd.read_csv(product_file, sep=';')
                #print(df.head())
                raw_dataframes[measure][section] = df

raw_dataframes['FF']['hist'].tail() # wind historic data sample

,STATIONS_ID,MESS_DATUM,QN_3,F,D,eor
639312,2014,2022123119,3,6.1,220,eor
639313,2014,2022123120,3,7.8,210,eor
639314,2014,2022123121,3,8.0,220,eor
639315,2014,2022123122,3,7.7,220,eor
639316,2014,2022123123,3,8.9,230,eor


In [15]:
import numpy as np

FLUGBAHN_DIR = 30 # 03 & 21

def get_wind_data():
    timeframe_start, timeframe_end, hour_of_day = 2021050100,2022083123,13

    df = raw_dataframes['FF']['hist']    

    df = df.drop(['STATIONS_ID','QN_3','eor'], axis=1).rename(columns={'   F': 'strength', '   D': 'direction'})
    mask = (df['MESS_DATUM'] >= timeframe_start) & (df['MESS_DATUM'] <= timeframe_end)
    #mask = mask & (df['strength'] >= 0) & (df['strength'] < 6) # too strong wind
    mask = mask & (df['MESS_DATUM'] % 100 == hour_of_day) & (df['direction'] >= 0)
    df = df[mask] # reduce size
    
    return df

def get_rain_data():
    timeframe_start, timeframe_end, hour_of_day = 2021050100,2022083123,13

    # R1;stdl. Niederschlagshoehe;mm;
    # RS_IND;Indikator Niederschlag ja/nein;numerischer Code;
    # WRTR;stdl. Niederschlagsform (=Niederschlagshoehe_ind);numerischer Code;

    df = raw_dataframes['RR']['hist']  
    df = df.drop(['STATIONS_ID','QN_8','eor','RS_IND','WRTR'], axis=1).rename(columns={'  R1':'percip'})
    mask = (df['MESS_DATUM'] >= timeframe_start) & (df['MESS_DATUM'] <= timeframe_end)
    mask = mask & (df['MESS_DATUM'] % 100 == hour_of_day)
    df = df[mask] # reduce size

    return df 

df_wind = get_wind_data()
df_rain = get_rain_data()

df_wind = df_wind.join(df_rain.set_index("MESS_DATUM"), on="MESS_DATUM", validate="1:1" )

print(df_wind.columns)
# 
df_wind['wind_alignment'] = df_wind.apply(lambda row: np.sin((row['direction'] - FLUGBAHN_DIR )* np.pi / 180.)**2 , axis=1)

df_wind.head()



Index(['MESS_DATUM', 'strength', 'direction', 'percip'], dtype='object')


,MESS_DATUM,strength,direction,percip,wind_alignment
624821,2021050113,4.6,270,0.0,0.969846
624845,2021050213,7.2,280,0.0,1.000000
624869,2021050313,6.2,260,0.0,0.883022
624893,2021050413,10.1,220,0.7,0.250000
624917,2021050513,10.5,260,0.0,0.883022
